In [42]:
pip install transformers torch 

Note: you may need to restart the kernel to use updated packages.Requirement already satisfied: transformers in e:\university\tlcn\chatbox\venv\lib\site-packages (4.43.4)



You should consider upgrading via the 'e:\university\TLCN\ChatBox\venv\Scripts\python.exe -m pip install --upgrade pip' command.


### 1. Import thư viện

In [2]:
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import pandas as pd


### 2. Load data

In [65]:

df = pd.read_csv('../csv/processed_medical.csv')
df


,Title,Detailed Content,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...
...,...,...,...
12483,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,dung_dịch keo ví_dụ tinh_bột hydroxyetyl album...,https://www.msdmanuals.com/vi-vn/professional/...
12484,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,máu thường được cung_cấp dưới dạng khối hồng_c...,https://www.msdmanuals.com/vi-vn/professional/...
12485,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,chất thay_thế máu là dịch mang oxy có_thể được...,https://www.msdmanuals.com/vi-vn/professional/...
12486,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,vì lượng nước_tiểu không cung_cấp chỉ từng phú...,https://www.msdmanuals.com/vi-vn/professional/...


##### Xử lý các giá trị null và duplicate

In [45]:
df.isnull().sum()

Title               0
Detailed Content    0
Reference Link      0
dtype: int64

In [46]:
# Giả sử df là DataFrame của bạn
# Kiểm tra số lượng giá trị trùng lặp trong cột 'Title'
duplicates_count_title = df['Title'].duplicated().sum()
duplicates_count_content = df['Detailed Content'].duplicated().sum()
print(f"Số lượng giá trị trùng lặp trong cột 'Title': {duplicates_count_title}")
print(f"Số lượng giá trị trùng lặp trong cột 'Detailed Content': {duplicates_count_content}")



Số lượng giá trị trùng lặp trong cột 'Title': 3283
Số lượng giá trị trùng lặp trong cột 'Detailed Content': 11380


In [47]:
# Tìm các title bị trùng lặp
duplicate_titles = df[df.duplicated(['Title'], keep=False)]

# Hàm để lấy từ ghép đầu tiên từ content
def get_first_compound_word(text):
    # Giả định rằng các từ ghép được phân tách bằng dấu '_'
    words = text.split()
    for word in words:
        if '_' in word:
            return word
    return ''

# Duyệt qua các title trùng lặp và cập nhật lại title
for index, row in duplicate_titles.iterrows():
    first_compound_word = get_first_compound_word(row['Detailed Content'])
    if first_compound_word:
        new_title = f"{row['Title']} theo {first_compound_word}"
        df.at[index, 'Title'] = new_title

# Kiểm tra lại số lượng giá trị trùng lặp sau khi cập nhật
duplicates_count_title_after = df['Title'].duplicated().sum()
print(f"Số lượng giá trị trùng lặp trong cột 'Title' sau khi cập nhật: {duplicates_count_title_after}")

Số lượng giá trị trùng lặp trong cột 'Title' sau khi cập nhật: 1286


In [48]:
df = df.drop_duplicates(subset=['Detailed Content'])

In [49]:
df = df.drop_duplicates()

# Tìm các giá trị trùng lặp

In [50]:
# Tìm các hàng trùng lặp
duplicates = df[df.duplicated(subset='Title', keep='first')]

# Hiển thị các hàng trùng lặp
print("Các hàng trùng lặp:")
duplicates

Các hàng trùng lặp:


,Title,Detailed Content,Reference Link
191,chống chỉ_định và thận_trọng khi tiêm về chống...,chống chỉ_định chính của vắc xin viêm gan b là...,https://www.msdmanuals.com/vi-vn/professional/...
192,chống chỉ_định và thận_trọng khi tiêm về thận_...,các thận_trọng chính vắc xin hepb là bệnh nhẹ ...,https://www.msdmanuals.com/vi-vn/professional/...
238,chống chỉ_định và thận_trọng khi tiêm về chống...,chống chỉ_định đối_với vắc xin ngừa rotavirus ...,https://www.msdmanuals.com/vi-vn/professional/...
345,chẩn_đoán nhiễm ký_sinh_trùng về huyết_thanh h...,huyết_thanh hoặc csf kháng_nguyên dùng để đánh...,https://www.msdmanuals.com/vi-vn/professional/...
426,lựa_chọn và sử_dụng kháng_sinh về quần_thể đặc...,one không có 1 – 2 g đường tĩnh_mạch 24 giờ mộ...,https://www.msdmanuals.com/vi-vn/professional/...
...,...,...,...
24990,mô_tả các bước_tiến_hành thủ_thuật về đặt ống ...,đẩy ống thông_qua dây_dẫn hướng đến bề_mặt da ...,https://www.msdmanuals.com/vi-vn/professional/...
24992,mô_tả các bước_tiến_hành thủ_thuật về sử_dụng ...,sử_dụng dẫn hướng siêu_âm trục ngắn lấy hình_ả...,https://www.msdmanuals.com/vi-vn/professional/...
25148,căn_nguyên của ngừng thở về tắc_nghẽn đường th...,tắc_nghẽn đường thở trên có_thể xảy ra ở trẻ s...,https://www.msdmanuals.com/vi-vn/professional/...
25272,sinh_lý bệnh ahrf về các nguyên_nhân ít gặp hơ...,các nguyên_nhân ít gặp hơn gây tổn_thương phổi...,https://www.msdmanuals.com/vi-vn/professional/...


### Xóa tài liệu tham khảo

In [51]:
# Lọc các tiêu đề có chứa 'Tài liệu tham khảo'
filtered_df = df[df['Title'].str.contains('tài_liệu tham_khảo', case=False, na=False)]

# In ra kết quả
filtered_df

,Title,Detailed Content,Reference Link
103,đánh_giá sốt không rõ nguyên_nhân về tài_liệu ...,1 lack of clinical utility of labeled white bl...,https://www.msdmanuals.com/vi-vn/professional/...
140,tiêm_chủng vắc xin về tài_liệu tham_khảo về vắ...,1 a needling problem shoulder injury related t...,https://www.msdmanuals.com/vi-vn/professional/...
171,an_toàn vắc xin về tài_liệu tham_khảo về an_to...,1 vaccines and autism a tale of shifting hypot...,https://www.msdmanuals.com/vi-vn/professional/...
195,chế_phẩm vắc xin hpv về tài_liệu tham_khảo về ...,1 human papillomavirus hpv vaccines 2019 1 nat...,https://www.msdmanuals.com/vi-vn/professional/...
269,chẩn_đoán tiêu_chảy do về tài_liệu tham_khảo c...,1 acg clinical guidelines prevention diagnosis...,https://www.msdmanuals.com/vi-vn/professional/...
...,...,...,...
25332,điều_trị nhiễm_trùng huyết và sốc nhiễm_trùng ...,1 identifying patients with sepsis on the hosp...,https://www.msdmanuals.com/vi-vn/professional/...
25346,sinh_lý bệnh của sốc về tài_liệu tham_khảo sin...,1 endothelial glycocalyx dysfunction in diseas...,https://www.msdmanuals.com/vi-vn/professional/...
25364,chẩn_đoán sốc về tài_liệu tham_khảo chẩn_đoán,1 image based resuscitation of the hypotensive...,https://www.msdmanuals.com/vi-vn/professional/...
25373,điều_trị sốc về tài_liệu tham_khảo về điều_trị,1 transfusion of plasma platelets and red bloo...,https://www.msdmanuals.com/vi-vn/professional/...


In [52]:
# Lọc các tiêu đề chứa 'Tài liệu tham khảo'
mask = df['Title'].str.contains('tài_liệu tham_khảo', case=False, na=False)

# Xóa các hàng chứa tiêu đề với 'Tài liệu tham khảo'
df = df[~mask]
df = df.reset_index(drop=True)

# In ra kết quả
df

,Title,Detailed Content,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...
...,...,...,...
12483,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,dung_dịch keo ví_dụ tinh_bột hydroxyetyl album...,https://www.msdmanuals.com/vi-vn/professional/...
12484,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,máu thường được cung_cấp dưới dạng khối hồng_c...,https://www.msdmanuals.com/vi-vn/professional/...
12485,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,chất thay_thế máu là dịch mang oxy có_thể được...,https://www.msdmanuals.com/vi-vn/professional/...
12486,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,vì lượng nước_tiểu không cung_cấp chỉ từng phú...,https://www.msdmanuals.com/vi-vn/professional/...


#### 3. Tìm câu trả lời 

In [30]:


# # Hàm tìm câu trả lời
# def find_answer_using_heuristic(question, context):
#     # Tạo ra các từ khóa từ câu hỏi, loại bỏ các từ dừng phổ biến
#     question_words = question.lower().split()
#     keywords = [word for word in question_words if word not in ENGLISH_STOP_WORDS and len(word) > 2]

#     if not keywords:
#         return "Câu hỏi không có từ khóa hợp lệ", 0, len("Câu hỏi không có từ khóa hợp lệ")

#     # Tạo một regex pattern từ các từ khóa
#     pattern = r'\b(' + '|'.join(map(re.escape, keywords)) + r')\b'

#     # Tìm kiếm các từ khóa trong context
#     matches = list(re.finditer(pattern, context.lower()))

#     if matches:
#         # Tính điểm cho các đoạn văn bản xung quanh vị trí tìm thấy từ khóa
#         best_match = max(matches, key=lambda match: sum(1 for word in keywords if word in context[match.start():match.end() + 100]))
#         answer_start = best_match.start()
#         answer_end = min(len(context), answer_start + 100)  # Giới hạn câu trả lời có độ dài tối đa 100 ký tự
#         return context[answer_start:answer_end], answer_start, answer_end
#     else:
#         # Nếu không tìm thấy, trả về câu trả lời mặc định
#         return "Câu trả lời không tìm thấy", 0, len("Câu trả lời không tìm thấy")

# # Ví dụ sử dụng
# titles = df['Title'].tolist()
# detailed_contents = df['Detailed Content'].tolist()

# answers = []
# answer_starts = []
# answer_ends = []

# for title, content in zip(titles, detailed_contents):
#     answer, start, end = find_answer_using_heuristic(title, content)
#     answers.append(answer)
#     answer_starts.append(start)
#     answer_ends.append(end)

# # Thêm các cột này vào DataFrame
# df['Answer'] = answers
# df['Answer Start'] = answer_starts
# df['Answer End'] = answer_ends

# # Kiểm tra kết quả
# df.head()

,Title,Detailed Content,Reference Link,Answer,Answer Start,Answer End
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...,Câu trả lời không tìm thấy,0,26
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...,Câu trả lời không tìm thấy,0,26
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...,đường hô_hấp có các hệ_thống lọc đường thở trê...,0,100
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...,rào_cản trong đường tiêu_hóa bao_gồm ph acid t...,4,104
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...,rào_cản về đường sinh_dục tiết_niệu bao_gồm độ...,4,104


In [57]:
# Data Preparation
titles = df['Title'].tolist()
detailed_contents = df['Detailed Content'].tolist()

# Đảm bảo tất cả các phần tử trong danh sách là chuỗi
titles = [str(title) for title in titles]
detailed_contents = [str(content) for content in detailed_contents]

In [58]:
import re

# Hàm tìm từ ghép có dấu '_'
def find_compound_words(texts):
    compound_words = set()
    for text in texts:
        # Tìm các từ chứa dấu '_'
        matches = re.findall(r'\b\w+_\w+\b', text)
        compound_words.update(matches)
    return list(compound_words)

# Tìm từ ghép trong titles và detailed_contents
compound_words_titles = find_compound_words(titles)
compound_words_contents = find_compound_words(detailed_contents)

# Kết hợp từ ghép từ cả titles và detailed_contents và đảm bảo tính duy nhất
compound_words = list(set(compound_words_titles + compound_words_contents))

# Kiểm tra số lượng từ ghép đã tìm thấy và các từ là duy nhất
print(f"Number of unique compound words: {len(compound_words)}")
print(compound_words[:10])  # In ra 10 từ ghép đầu tiên để kiểm tra

Number of unique compound words: 10739
['đút_nút', 'căng_giãn', 'khởi_phát_bệnh', 'không_gỉ', 'cuồng_dâm', 'tâm_thất_mạn', 'candida_miệng', 'tống_phân', 'năng_lực', 'chuyển_hóa_fructose']


In [59]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

# Tải mô hình và tokenizer
model_name = "PhucDanh/Bartpho-fine-tuning-model-for-question-answering"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [60]:
num_added_tokens = tokenizer.add_tokens(compound_words)

model.resize_token_embeddings(len(tokenizer))

print(f"Added {num_added_tokens} tokens. New vocab size: {len(tokenizer)}")

Added 10739 tokens. New vocab size: 50770


In [62]:
from tqdm import tqdm

def find_answer_with_model(question, context):
    # Tokenize the input
    inputs = tokenizer(question, context, return_tensors='pt', truncation=True, padding=True)

    # BART không yêu cầu 'token_type_ids', nên không cần truyền tham số này
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Forward pass
    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)

    # Get answer from model outputs
    start_idx = torch.argmax(outputs.start_logits)
    end_idx = torch.argmax(outputs.end_logits) + 1

    # Decode the answer
    answer_tokens = tokenizer.convert_ids_to_tokens(input_ids[0][start_idx:end_idx])
    answer = tokenizer.convert_tokens_to_string(answer_tokens)
    
    return answer, start_idx.item(), end_idx.item()

# Ví dụ sử dụng với DataFrame
titles = df['Title'].tolist()
detailed_contents = df['Detailed Content'].tolist()

answers = []
answer_starts = []
answer_ends = []

# Sử dụng tqdm để hiển thị tiến độ
for title, content in tqdm(zip(titles, detailed_contents), total=len(titles), desc="Processing"):
    answer, start, end = find_answer_with_model(title, content)
    answers.append(answer)
    answer_starts.append(start)
    answer_ends.append(end)

# Thêm các cột này vào DataFrame
df['Answer'] = answers
df['Answer Start'] = answer_starts
df['Answer End'] = answer_ends

# Kiểm tra kết quả
df.head()




Processing:   0%|          | 25/12488 [00:56<7:51:24,  2.27s/it] 


KeyboardInterrupt: 

In [31]:
# Tìm các hàng trùng lặp với hàng số 5 (index = 4)
duplicate_rows = df[df['Title'] == df.loc[5504, 'Title']]

# Hiển thị kết quả
duplicate_rows

,Title,Detailed Content,Reference Link,Answer,Answer Start,Answer End
5504,chống chỉ_định với chọc dò dịch màng phổi về c...,chống chỉ_định tương_đối bệnh_lý chảy_máu hoặc...,https://www.msdmanuals.com/vi-vn/professional/...,chống chỉ_định tương_đối bệnh_lý chảy_máu hoặc...,0,100


# Lưu data

In [63]:
# Lưu DataFrame vào file CSV
df.to_csv('../csv/processed_medical.csv', index=False, encoding='utf-8')

# Show detail dataset

In [64]:
df

,Title,Detailed Content,Reference Link
0,rào_cản tự_nhiên chống lại nhiễm_trùng về da,da thường chặn các vi_sinh_vật xâm_nhập trừ kh...,https://www.msdmanuals.com/vi-vn/professional/...
1,rào_cản tự_nhiên chống lại nhiễm_trùng về niêm...,nhiều màng_nhầy ngập trong chất tiết có đặc_tí...,https://www.msdmanuals.com/vi-vn/professional/...
2,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,đường hô_hấp có các hệ_thống lọc đường thở trê...,https://www.msdmanuals.com/vi-vn/professional/...
3,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản trong đường tiêu_hóa bao_gồm ph ac...,https://www.msdmanuals.com/vi-vn/professional/...
4,rào_cản tự_nhiên chống lại nhiễm_trùng về đườn...,các rào_cản về đường sinh_dục tiết_niệu bao_gồ...,https://www.msdmanuals.com/vi-vn/professional/...
...,...,...,...
12483,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,dung_dịch keo ví_dụ tinh_bột hydroxyetyl album...,https://www.msdmanuals.com/vi-vn/professional/...
12484,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,máu thường được cung_cấp dưới dạng khối hồng_c...,https://www.msdmanuals.com/vi-vn/professional/...
12485,dịch để hồi_sức bằng dịch qua đường tĩnh_mạch ...,chất thay_thế máu là dịch mang oxy có_thể được...,https://www.msdmanuals.com/vi-vn/professional/...
12486,tiêu_chí đánh_giá và theo_dõi hồi_sức bằng dịc...,vì lượng nước_tiểu không cung_cấp chỉ từng phú...,https://www.msdmanuals.com/vi-vn/professional/...


In [39]:
# Lấy nội dung của cột "Content" ở dòng đầu tiên
first_row_title = df.loc[200, 'Title']
first_row_content = df.loc[200, 'Detailed Content']
first_row_answer = df.loc[200, 'Answer']

# In nội dung của cột "Content" ở dòng đầu tiên
print("Content of the first row:")
print(first_row_title)    
print(first_row_content)    
print(first_row_answer)    

Content of the first row:
điều_trị uốn_ván về chăm_sóc vết_thương
bụi bẩn và mô chết thúc_đẩy c_tetani tăng_trưởng nhanh_chóng nên việc mở ở ổ vết_thương đặc_biệt là vết_thương đâm sâu là rất cần_thiết thuốc kháng_sinh không phải là chất thay_thế cho quá_trình khử_trùng và chủng ngừa đầy_đủ nhưng thường được sử_dụng
vết_thương đặc_biệt là vết_thương đâm sâu là rất cần_thiết thuốc kháng_sinh không phải là chất thay_


In [36]:
def print_max_length_rows(df, columns):
    max_length_info = {}
    
    for column in columns:
        # Tính toán chiều dài của từng văn bản trong cột
        df[f'{column}_length'] = df[column].apply(lambda x: len(str(x)))  # Thêm cột độ dài
        
        # Tìm hàng có độ dài tối đa
        max_length_row = df.loc[df[f'{column}_length'].idxmax()]
        
        # Lưu thông tin vào từ điển
        max_length_info[column] = {
            'index': max_length_row.name,
            'length': max_length_row[f'{column}_length'],
            'value': max_length_row[column]
        }
    
    # In kết quả cuối cùng
    for column, info in max_length_info.items():
        print(f"Max length in '{column}':")
        print(f"Index: {info['index']}")
        print(f"Length: {info['length']}")
        print(f"Value: {info['value']}")
        print('-' * 40)

# Xác định các cột cần kiểm tra
columns_to_check = ['Title', 'Detailed Content']

# In hàng có độ dài tối đa trong từng cột
print_max_length_rows(df, columns_to_check)

Max length in 'Title':
Index: 99
Length: 248
Value: liều_lượng và cách sử_dụng vắc xin phòng covid 19 về đối_với các loại vắc xin ngừa covid 19 có sẵn để sử_dụng ở hoa_kỳ tất_cả thông_tin về liều_lượng và cách sử_dụng cho mọi nhóm tuổi và những người có tình_trạng đặc_biệt đều có sẵn từ cdc tại phần
----------------------------------------
Max length in 'Detailed Content':
Index: 8420
Length: 4044
Value: đái_tháo_đường có_thể là lý_do chính khiến bệnh_nhân phải nhập_viện hoặc có_thể đi kèm với các bệnh khác cần được điều_trị nội_trú tất_cả bệnh_nhân nhiễm toan ceton do đái_tháo_đường tình_trạng tăng đường_huyết tăng thẩm_thấu hoặc hạ đường_huyết kéo_dài hoặc nặng cần phải được nhập_viện bệnh_nhân với hạ đường_huyết do sulfonylureas tăng đường_huyết kiểm_soát kém hoặc biến_chứng đái_tháo đường cấp_tính nặng có_thể hưởng lợi từ nhập_viện trẻ_em và thiếu_niên với đái_tháo_đường mới khởi_phát cũng có_thể có lợi từ nhập_viện kiếm soát có_thể tồi_tệ hơn khi xuất_viện phác_đồ insulin đã được 